# One Period Binomial Model Call Option Price Prediction

### Shantanu Laghate

### Hacker Hour - Financial Programming (Part 1)

### Feb 15, 2019

Model calibrated on 3 TSLA calls on Feb 6, 2018 with Strikes 310, 315, 325 with expiry on Mar 22, 2019

In [17]:
import numpy as np

# Set Initial Conditions
3 TSLA options with expiry Mar 22, 2019 (~2 months from now), with the current stock price at $319.02. 
Ki, V0i represents the strike price and the present market ask price for each option.

In [30]:
T = 1/6.0
S0 = 319.02
# Option 1: 280 call, $33.10
# Option 2: 275 call, $35.95
# Option 3: 270 call, $39.00

K1 = 315.00
V01 = 24.00

K2 = 310.00
V02 = 26.90

K3 = 325.00
V03 = 18.65

# Define Pricing Functions

For a call option, V(u) = max(S0*u - K, 0), and V(d) = max(S0*d - K, 0). We substitute this into the one-period pricing binomial model, and subtract V_0 from the prediction to find the difference. Ideally, u, d, and r should be set so all 3 of these differences will be 0. 

In [31]:
def e1(u, d, r, strike, realprice):
    Vu = max(S0*u - strike, 0)
    Vd = max(S0*d - strike, 0)
    ert = np.exp(-1*r*T)
    p = (ert - d)/(u-d)
    
    return ert*(Vu*p + Vd*(1-p)) - realprice

# Solving for u, d, r

All symbolic toolbox methods of solving this system didn't work (highly nonlinear and unusual), so I resorted to iteration through possible values. I set the threshold to 0.1 (meaning all three predicted prices must be within 10 cents of market price), and tried 100 values between [0.5, 1.5] for u and d, and [0, 0.03] for r. More than 100 values would have provided a more accurate result, but was running for too long. 

There were a few values that matched, and I fixed u = 1.176 and d = 0.7424. Since the r value varied widely, the next step was to find a more accurate value.

In [34]:
threshold = 0.1
for i in np.linspace(1, 1.5, 100):
    for j in np.linspace(0.5, 1, 100):
        for k in np.linspace(0.01, 0.03, 100):
            if (i > j 
                and abs(e1(i, j, k, K1, V01)) < threshold 
                and abs(e1(i, j, k, K2, V02)) < threshold 
                and abs(e1(i, j, k, K3, V03)) < threshold):
                
                print(i, j, k)
  

1.1262626262626263 0.8383838383838385 0.026161616161616157
1.1262626262626263 0.8383838383838385 0.02636363636363636
1.1262626262626263 0.8383838383838385 0.026565656565656563
1.1262626262626263 0.8434343434343434 0.014242424242424242
1.1262626262626263 0.8434343434343434 0.014444444444444444
1.1262626262626263 0.8434343434343434 0.014646464646464647


# Finding a precise r value

We can again iterate through possible r values to find what results in a close fit. For the final answer, I found that r=0.0142 was a good average.

In [38]:
# from here, we find that u = 1.1767, d = 0.7424. Now we must find the most exact implied r:
threshold = 0.1 # somewhat arbitrary (smaller values didn't give any results)
U = 1.1262
D = 0.8383
sums = 0
count = 0
for k in np.linspace(0.01, 0.03, 1000):
    if (abs(e1(U, D, k, K1, V01)) < threshold 
        and abs(e1(U, D, k, K2, V02)) < threshold 
        and abs(e1(U, D, k, K3, V03)) < threshold):
        sums += k
        count += 1
print("Average:", sums/count)
R = sums/count

Average: 0.02634634634634634


# Call option price predictor function
We have now calibrated our model for values of u, d, and r. Assuming S0 and T stay the same, we can now calculate the predicted prices for different strike prices using the binomial model. A sample list of values is given below:

In [39]:
## Let's take r = 0.0142, the average value from the above list.
#R = 0.0265
def call_predict(strike):
    Vu = max(S0*U - strike, 0)
    Vd = max(S0*D - strike, 0)
    ert = np.exp(-1*R*T)
    p = (ert - D)/(U-D)
    
    return ert*(Vu*p + Vd*(1-p))

for i in np.arange(200, 400, 10):
    print(i, call_predict(i))

200 117.10688211700153
210 107.15069642826046
220 97.19451073951939
230 87.23832505077831
240 77.28213936203723
250 67.32595367329615
260 57.3697679845551
270 48.572120931643425
280 43.13171494346136
290 37.69130895527931
300 32.250902967097254
310 26.810496978915197
320 21.37009099073314
330 15.929685002551082
340 10.489279014369027
350 5.048873026186969
360 0.0
370 0.0
380 0.0
390 0.0
